In [2]:
import pandas as pd
import re
import os

#Recoleccion de tokens historical


In [484]:
import requests
import pandas as pd

# Definir la URL de la API para obtener la lista de criptomonedas
listings_url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical'

# Definir los parámetros de la solicitud para obtener la lista de criptomonedas
listings_parameters = {
  'date': '2024-04-29',  # Fecha de inicio de los datos históricos
  'convert': 'USD',             # Moneda en la que se convertirán los valores
  'cryptocurrency_type':'tokens',
  'limit': 1000                # Ajusta el límite para asegurarte de obtener todas las criptomonedas
}

# Definir los headers con la clave de API
headers = {
  'X-CMC_PRO_API_KEY': '83a12ab3-3579-4070-ab2f-e86210b78037'  # Reemplazar 'TU_CLAVE_DE_API' con tu clave de API
}

# Realizar la solicitud GET a la API para obtener la lista de criptomonedas
response = requests.get(listings_url, headers=headers, params=listings_parameters)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa, obtener los datos de respuesta
    data = response.json()

    # Inicializar un DataFrame vacío para almacenar los datos
    data_token = pd.DataFrame()

    # Iterar sobre cada criptomoneda en la lista
    for cryptocurrency in data['data']:
        # Normalizar la data del token y guardar en un DataFrame
        df_tokens = pd.json_normalize(cryptocurrency)

        # Concatenar el DataFrame del token al DataFrame principal
        data_token = pd.concat([data_token, df_tokens], ignore_index=True)

    # Imprimir el DataFrame resultante
    print(data_token)
else:
    # La solicitud falló, imprimir el mensaje de error
    print("Error al obtener la lista de criptomonedas:", response.status_code, response.text)


        id               name symbol               slug  num_market_pairs  \
0      825        Tether USDt   USDT             tether             84370   
1     3408               USDC   USDC           usd-coin             18752   
2     5994          Shiba Inu   SHIB          shiba-inu               802   
3     1975          Chainlink   LINK          chainlink              1739   
4     8916  Internet Computer    ICP  internet-computer               236   
..     ...                ...    ...                ...               ...   
995  28220               Soil   SOIL               soil                15   
996  29185             Amulet    AMU             amulet                 9   
997  19689           Hydranet    HDN           hydranet                26   
998   4197         ShareToken    SHR         sharetoken                16   
999   4167        Bitrue Coin    BTR        bitrue-coin                23   

                   date_added  \
0    2015-02-25T00:00:00.000Z   
1    2018

In [526]:
data_token.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                1000 non-null   int64  
 1   name                              1000 non-null   object 
 2   symbol                            1000 non-null   object 
 3   slug                              1000 non-null   object 
 4   num_market_pairs                  1000 non-null   int64  
 5   date_added                        1000 non-null   object 
 6   tags                              1000 non-null   object 
 7   circulating_supply                1000 non-null   float64
 8   total_supply                      1000 non-null   float64
 9   infinite_supply                   1000 non-null   bool   
 10  cmc_rank                          1000 non-null   int64  
 11  self_reported_circulating_supply  571 non-null    float64
 12  self_re

In [485]:
import os

# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data/raw'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"data_tokens_{'2024-04-29'}.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
data_token.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/raw/data_tokens_2024-04-29.csv


#Seleccion y categorizacion

In [500]:
dia = '2024-04-29'

In [501]:
data_1 = pd.read_csv(f'/content/data/raw/data_tokens_{dia}.csv')

In [502]:
data_1.info() #id,name,symbol,tags,quote.USD.price,1000 non-null   float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                1000 non-null   int64  
 1   name                              1000 non-null   object 
 2   symbol                            1000 non-null   object 
 3   slug                              1000 non-null   object 
 4   num_market_pairs                  1000 non-null   int64  
 5   date_added                        1000 non-null   object 
 6   tags                              1000 non-null   object 
 7   circulating_supply                1000 non-null   float64
 8   total_supply                      1000 non-null   float64
 9   infinite_supply                   1000 non-null   bool   
 10  cmc_rank                          1000 non-null   int64  
 11  self_reported_circulating_supply  571 non-null    float64
 12  self_re

In [527]:
data_1_selected = data_1[['id', 'name', 'symbol', 'tags', 'quote.USD.price','quote.USD.percent_change_24h']]
specific_date = pd.to_datetime(dia)
data_1_selected.insert(0, 'date', specific_date)

In [528]:
data_detallada =  data_1_selected
data_portafolio = data_1_selected

#Busqueda de categorias

In [ ]:
data_detallada['IA'] = 0
data_detallada['Gaming'] = 0
data_detallada['RWA'] = 0
data_detallada['Meme'] = 0

In [531]:
keywords = {
    'IA': ['AI & Big Data'],
    'Gaming': ['Gaming'],
    'RWA': ['Real World Assets'],
    'Meme': ['Meme']
}

In [532]:
# Diccionario que mapea palabras clave a sus respectivos números
keyword_mapping = {'ai-big-data': 1,'gaming': 2, 'real-world-assets': 3,'memes': 4,}

In [533]:
# Iterar sobre las filas del DataFrame
for index, row in data_detallada.iterrows():
    tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
    tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista

    found_keywords = set()  # Conjunto para registrar qué palabras clave se encuentran

    # Buscar las palabras clave y agregarlas al conjunto
    for keyword in tags_list:
        if keyword in keyword_mapping:
            found_keywords.add(keyword_mapping[keyword])

    # Asignar los valores correspondientes a las columnas "IA", "Gaming", "RWA" y "Meme"
    for col_name, keyword_num in {'IA': 1, 'Gaming': 2, 'RWA': 3, 'Meme': 4}.items():
        data_detallada.at[index, col_name] = 1 if keyword_num in found_keywords else 0

In [535]:
data_detallada

,date,id,name,symbol,tags,quote.USD.price,quote.USD.percent_change_24h,IA,Gaming,RWA,Meme
0,2024-04-29,825,Tether USDt,USDT,"['stablecoin', 'asset-backed-stablecoin', 'ava...",0.999713,-0.001911,0,0,0,0
1,2024-04-29,3408,USDC,USDC,"['medium-of-exchange', 'stablecoin', 'asset-ba...",1.000011,-0.005309,0,0,0,0
2,2024-04-29,5994,Shiba Inu,SHIB,"['memes', 'ethereum-ecosystem', 'doggone-dogge...",0.000024,-0.438455,0,0,0,1
3,2024-04-29,1975,Chainlink,LINK,"['platform', 'defi', 'oracles', 'smart-contrac...",14.073785,1.924352,0,0,1,0
4,2024-04-29,8916,Internet Computer,ICP,"['platform', 'distributed-computing', 'collect...",13.445103,-0.791537,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
995,2024-04-29,28220,Soil,SOIL,"['defi', 'seedify', 'real-world-assets']",1.798528,-11.770054,0,0,1,0
996,2024-04-29,29185,Amulet,AMU,"['insurance', 'solana-ecosystem']",0.029286,-2.891448,0,0,0,0
997,2024-04-29,19689,Hydranet,HDN,['arbitrum-ecosytem'],0.038783,3.479685,0,0,0,0
998,2024-04-29,4197,ShareToken,SHR,[],0.002586,1.774144,0,0,0,0


#Busqueda de portfolios

In [536]:

# Lista para almacenar los portfolios encontrados únicos
unique_portfolios = []

# Iterar sobre las filas del DataFrame
for index, row in data_1_selected.iterrows():
    tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
    tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista

    # Buscar palabras que contengan "portfolio" en la lista de tags
    for tag in tags_list:
        if re.search(r'\b\w+-portfolio\b', tag):
            # Agregar el portfolio encontrado a la lista si no está presente
            if tag not in unique_portfolios:
                unique_portfolios.append(tag)

# Imprimir la lista de portfolios encontrados únicos
print(unique_portfolios)

['coinbase-ventures-portfolio', 'framework-ventures-portfolio', 'polychain-capital-portfolio', 'electric-capital-portfolio', 'fenbushi-capital-portfolio', 'hashkey-capital-portfolio', 'a16z-portfolio', 'multicoin-capital-portfolio', 'exnetwork-capital-portfolio', 'kenetic-capital-portfolio', 'alameda-research-portfolio', 'three-arrows-capital-portfolio', 'blockchain-capital-portfolio', 'defiance-capital-portfolio', 'pantera-capital-portfolio', 'paradigm-portfolio', 'dwf-labs-portfolio', 'arrington-xrp-capital-portfolio', 'egirl-capital-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', '1confirmation-portfolio', 'placeholder-ventures-portfolio', 'standard-crypto-portfolio', 'coinfund-portfolio', 'fabric-ventures-portfolio', 'ledgerprime-portfolio', 'binance-labs-portfolio', 'cms-holdings-portfolio', 'okx-ventures-portfolio', 'animoca-brands-portfolio', 'boostvc-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'winklevoss-capital-portfolio', 'vbc-ventures-port

In [537]:
# Crear un diccionario para almacenar el conteo de tokens por portfolio
conteo_tokens_por_portfolio = {}

# Iterar sobre cada portafolio único
for portfolio in unique_portfolios:
    # Contador para almacenar el conteo de tokens para este portafolio
    conteo_tokens = 0

    # Iterar sobre las filas del DataFrame
    for index, row in data_1_selected.iterrows():
        tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
        tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista
        # Verificar si el portafolio está presente en la lista de tags
        if portfolio in tags_list:
            conteo_tokens += 1  # Incrementar el conteo de tokens
    # Almacenar el conteo de tokens para este portafolio en el diccionario
    conteo_tokens_por_portfolio[portfolio] = conteo_tokens

# Imprimir el conteo de tokens por portfolio
for portfolio, conteo in conteo_tokens_por_portfolio.items():
    print(f"El portfolio '{portfolio}' tiene {conteo} tokens.")


El portfolio 'coinbase-ventures-portfolio' tiene 29 tokens.
El portfolio 'framework-ventures-portfolio' tiene 10 tokens.
El portfolio 'polychain-capital-portfolio' tiene 16 tokens.
El portfolio 'electric-capital-portfolio' tiene 6 tokens.
El portfolio 'fenbushi-capital-portfolio' tiene 3 tokens.
El portfolio 'hashkey-capital-portfolio' tiene 8 tokens.
El portfolio 'a16z-portfolio' tiene 12 tokens.
El portfolio 'multicoin-capital-portfolio' tiene 14 tokens.
El portfolio 'exnetwork-capital-portfolio' tiene 28 tokens.
El portfolio 'kenetic-capital-portfolio' tiene 23 tokens.
El portfolio 'alameda-research-portfolio' tiene 34 tokens.
El portfolio 'three-arrows-capital-portfolio' tiene 20 tokens.
El portfolio 'blockchain-capital-portfolio' tiene 11 tokens.
El portfolio 'defiance-capital-portfolio' tiene 10 tokens.
El portfolio 'pantera-capital-portfolio' tiene 33 tokens.
El portfolio 'paradigm-portfolio' tiene 10 tokens.
El portfolio 'dwf-labs-portfolio' tiene 97 tokens.
El portfolio 'arrin

#Data detallada


In [538]:
import pandas as pd

# Crear un DataFrame nuevo para almacenar los datos
nueva_data = pd.DataFrame(columns=['date', 'portfolio', 'nombre_token', 'symbol_token', 'IA', 'Gaming', 'RWA', 'Meme'])

# Iterar sobre el diccionario conteo_tokens_por_portfolio y agregar los datos al DataFrame nuevo
for portfolio, conteo in conteo_tokens_por_portfolio.items():
    # Crear una fila para cada token en este portfolio y agregarla al DataFrame nuevo
    for index, row in data_detallada.iterrows():
        tags_str = row['tags']
        tags_list = eval(tags_str)

        if portfolio in tags_list:
             # Agregar la información al DataFrame nuevo
            nueva_data.loc[len(nueva_data)] =[row['date'],
                                             portfolio,
                                            row['name'],
                                            row['symbol'],
                                            row['IA'],
                                            row['Gaming'],
                                            row['RWA'],
                                            row['Meme']]
            # Si ya se han agregado todos los tokens para este portfolio, detener la iteración
            if conteo == 0:
                break
            else:
                conteo -= 1


In [539]:
# Imprimir las primeras filas del nuevo DataFrame para verificar
nueva_data

,date,portfolio,nombre_token,symbol_token,IA,Gaming,RWA,Meme
0,2024-04-29,coinbase-ventures-portfolio,USDC,USDC,0,0,0,0
1,2024-04-29,coinbase-ventures-portfolio,Uniswap,UNI,0,0,0,0
2,2024-04-29,coinbase-ventures-portfolio,The Graph,GRT,1,0,0,0
3,2024-04-29,coinbase-ventures-portfolio,Sui,SUI,0,0,0,0
4,2024-04-29,coinbase-ventures-portfolio,Wormhole,W,0,0,0,0
...,...,...,...,...,...,...,...,...
595,2024-04-29,crypto-com-capital-portfolio,Artificial Liquid Intelligence,ALI,1,0,0,0
596,2024-04-29,poolz-finance-portfolio,SingularityDAO,SDAO,1,0,0,0
597,2024-04-29,poolz-finance-portfolio,HyperGPT,HGPT,1,0,0,0
598,2024-04-29,poolz-finance-portfolio,HAPI Protocol,HAPI,0,0,0,0


In [513]:
# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data/portafolios'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"data_portafolios_{dia}.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
nueva_data.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/portafolios/data_portafolios_2024-04-29.csv


#Combinar csv portafolios

In [514]:
import os
import pandas as pd
import glob

# Directorio donde se encuentran los archivos CSV
raw_data_directory = 'data/portafolios'

# Obtener la lista de todos los archivos CSV en el directorio
csv_files = glob.glob(os.path.join(raw_data_directory, '*.csv'))

# Lista para almacenar los DataFrames de los archivos CSV
data_frames = []

# Iterar sobre los archivos CSV y leer cada uno como un DataFrame
for csv_file in csv_files:
    data_frames.append(pd.read_csv(csv_file))

# Concatenar todos los DataFrames en uno solo
data_2 = pd.concat(data_frames, ignore_index=True)


In [515]:
data_2

,date,portfolio,nombre_token,symbol_token,IA,Gaming,RWA,Meme
0,2024-04-16,coinbase-ventures-portfolio,USDC,USDC,0,0,0,0
1,2024-04-16,coinbase-ventures-portfolio,Uniswap,UNI,0,0,0,0
2,2024-04-16,coinbase-ventures-portfolio,The Graph,GRT,1,0,0,0
3,2024-04-16,coinbase-ventures-portfolio,Sui,SUI,0,0,0,0
4,2024-04-16,coinbase-ventures-portfolio,Wormhole,W,0,0,0,0
...,...,...,...,...,...,...,...,...
19730,2024-04-27,crypto-com-capital-portfolio,Artificial Liquid Intelligence,ALI,1,0,0,0
19731,2024-04-27,poolz-finance-portfolio,SingularityDAO,SDAO,1,0,0,0
19732,2024-04-27,poolz-finance-portfolio,HyperGPT,HGPT,1,0,0,0
19733,2024-04-27,poolz-finance-portfolio,HAPI Protocol,HAPI,0,0,0,0


In [516]:
# Ordenar por fecha
data_2 = data_2.sort_values(by='date')

In [518]:
data_2 =data_2.reset_index(drop=True)
data_2

,date,portfolio,nombre_token,symbol_token,IA,Gaming,RWA,Meme
0,2024-03-28,poolz-finance-portfolio,Gains,GAINS,0,0,0,0
1,2024-03-28,pantera-capital-portfolio,Compound,COMP,0,0,0,0
2,2024-03-28,pantera-capital-portfolio,FTX Token,FTT,0,0,0,0
3,2024-03-28,pantera-capital-portfolio,0x Protocol,ZRX,0,0,0,0
4,2024-03-28,pantera-capital-portfolio,Oasis Network,ROSE,1,0,0,0
...,...,...,...,...,...,...,...,...
19730,2024-04-29,blockchain-capital-portfolio,Uniswap,UNI,0,0,0,0
19731,2024-04-29,blockchain-capital-portfolio,Aave,AAVE,0,0,0,0
19732,2024-04-29,blockchain-capital-portfolio,Worldcoin,WLD,0,0,0,0
19733,2024-04-29,blockchain-capital-portfolio,1inch Network,1INCH,0,0,0,0


#Data portafolio

In [ ]:
import pandas as pd

# Convertir la columna 'date' a tipo datetime si aún no lo está
data_detallada['date'] = pd.to_datetime(data_detallada['date'])

# Especificar el rango de fechas
fecha_inicio = '2024-03-28'
fecha_fin = '2024-04-29'

# Crear una lista para almacenar los resultados
resultados = []

# Iterar sobre cada fecha en el rango
for fecha in pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D'):
    # Iterar sobre cada nombre de portfolio único
    for nombre_portfolio in unique_portfolios:
        # Filtrar los datos para la fecha y el portfolio específicos
        filtro = (data_detallada['date'] == fecha) & (data_detallada['portfolio'] == nombre_portfolio)
        datos_filtrados = data_detallada[filtro]

        # Contar el número de tokens en el portfolio para la fecha específica
        conteo_tokens = datos_filtrados.shape[0]

        # Guardar los resultados en la lista
        resultados.append({'Fecha': fecha, 'Nombre Portfolio': nombre_portfolio, 'Tokens Encontrados': conteo_tokens})

# Crear un DataFrame a partir de la lista de resultados
data_resultados = pd.DataFrame(resultados)

# Imprimir los resultados
data_resultados


,Fecha,Nombre Portfolio,Tokens Encontrados
0,2024-03-28,coinbase-ventures-portfolio,28
1,2024-03-28,framework-ventures-portfolio,10
2,2024-03-28,polychain-capital-portfolio,15
3,2024-03-28,electric-capital-portfolio,6
4,2024-03-28,fenbushi-capital-portfolio,3
...,...,...,...
1414,2024-04-29,polygon-ventures-portfolio,1
1415,2024-04-29,usv-portfolio,1
1416,2024-04-29,circle-ventures-portfolio,3
1417,2024-04-29,crypto-com-capital-portfolio,1


In [ ]:
# Filtrar el DataFrame por la fecha específica
data_tokens_fecha_especifica = data_1[data_1['fecha'] == data_1]

# Calcular el ranking por aveg_change_24h
data_tokens_fecha_especifica['ranking'] = data_tokens_fecha_especifica['quote.USD.percent_change_24h'].rank(ascending=False)

# Ordenar el DataFrame por aveg_change_24h de manera descendente
data_tokens_fecha_especifica = data_tokens_fecha_especifica.sort_values(by='quote.USD.percent_change_24h', ascending=False)


In [ ]:
data_portafolio = pd.read_csv('/content/data/Dataset-portafolio.csv')

In [ ]:
data_portafolio.info()
data_portafolio.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                41 non-null     object
 1   name              41 non-null     object
 2   num_tokens        41 non-null     int64 
 3   token_ganadores   41 non-null     int64 
 4   token_perdedores  41 non-null     int64 
 5   ranking           41 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.0+ KB


(41, 6)

In [ ]:
data_portafolio.head()

,id,name,num_tokens,token_ganadores,token_perdedores,ranking
0,655c563a19d020516fba113a,eGirl Capital Portfolio,12,7,3,7
1,605e2a4ed41eae1066535f72,DeFiance Capital Portfolio,15,12,2,10
2,605e2ec9d41eae1066535f82,Paradigm Portfolio,17,14,2,17
3,605e2a78d41eae1066535f73,Coinfund Portfolio,15,13,2,26
4,604f2775ebccdd50cd175fdb,Polychain Capital Portfolio,31,27,2,28


#Configuraciones

In [519]:
# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"Dataset-portafolio-detallado.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
data_2.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/Dataset-portafolio-detallado.csv


In [ ]:
from google.colab import files
import shutil

# Selecciona el archivo CSV desde tu sistema local
uploaded = files.upload()
# Ruta del archivo CSV cargado en Colab
csv_file_path = '/content/Dataset-portafolio.csv'  # Cambia 'nombre_del_archivo.csv' al nombre real de tu archivo
# Ruta de la carpeta de destino
destination_folder = 'data'  # Cambia 'carpeta_de_destino' al nombre de tu carpeta de destino

# Mover el archivo CSV a la carpeta de destino
shutil.move(csv_file_path, destination_folder)


Saving Dataset-portafolio.csv to Dataset-portafolio (1).csv


'data/Dataset-portafolio.csv'

In [ ]:
shutil.rmtree('/content/data/portafolios(1)')

In [542]:
import os
import shutil
import zipfile

# Carpeta que deseas comprimir
folder_to_compress = '/content/data'  # Cambia 'carpeta_a_comprimir' al nombre de la carpeta que deseas comprimir

# Comprimir la carpeta en un archivo RAR
output_zip_filename = 'Dataset.zip'
with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_compress):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_compress))

# Descargar el archivo ZIP
from google.colab import files
files.download(output_zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import zipfile
with zipfile.ZipFile("/content/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/data")

In [ ]:
!rm -rf "/content/raw"

#Data Detallada requerida

In [540]:
data_detallada = pd.read_csv('/content/data/Dataset-portafolio-detallado.csv')

In [541]:
data_detallada.info()
data_detallada.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          19735 non-null  object
 1   portfolio     19735 non-null  object
 2   nombre_token  19735 non-null  object
 3   symbol_token  19735 non-null  object
 4   IA            19735 non-null  int64 
 5   Gaming        19735 non-null  int64 
 6   RWA           19735 non-null  int64 
 7   Meme          19735 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 1.2+ MB


(19735, 8)

In [38]:
data_detallada.head()

,date,portfolio,nombre_token,symbol_token,IA,Gaming,RWA,Meme
0,2024-03-27,boostvc-portfolio,Decentraland,MANA,0,1,0,0
1,2024-03-27,coinfund-portfolio,The Graph,GRT,1,0,0,0
2,2024-03-27,dwf-labs-portfolio,Mantle,MNT,0,0,0,0
3,2024-03-27,arrington-xrp-capital-portfolio,Immutable,IMX,0,1,0,0
4,2024-03-27,egirl-capital-portfolio,Arbitrum,ARB,0,0,0,0
